In [15]:
import pymysql

In [16]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [17]:
ssc = StreamingContext(sc, 1)


In [18]:
ssc.checkpoint("file:///home/lygbug666/workdir/streaming/")

In [19]:
initialStateRDD = sc.parallelize([(u'hello', 1), (u'world', 1)])

In [20]:
def updateFunc(new_values, last_sum):
            return sum(new_values) + (last_sum or 0)

In [21]:
hostname = "localhost"
port = 9999
lines = ssc.socketTextStream(hostname, port)

In [22]:
running_counts = lines.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).updateStateByKey(updateFunc, initialRDD=initialStateRDD)
running_counts.pprint()

In [23]:
def dbfunc(records):
        db = pymysql.connect("localhost","root","123","spark")
        cursor = db.cursor()
        def doinsert(p):
            sql = "insert into wordcount(word,count) values ('%s', '%s')" % (str(p[0]), str(p[1]))
            try:
                cursor.execute(sql)                
                db.commit()
            except:                    
                db.rollback()
 
        for item in records:
            doinsert(item)

In [24]:
def func(rdd):
        repartitionedRDD = rdd.repartition(3)
        repartitionedRDD.foreachPartition(dbfunc)

In [25]:
running_counts.foreachRDD(func)

In [14]:
ssc.start()
sc.awaitTermination()

AttributeError: 'SparkContext' object has no attribute 'start'